# Things to do

Now is a good time to look at everythign we have - here is the file location:

/Users/yd211/Documents/GitHub/DataSet/DVSCrashData/MN Crash Data 2015 and Documentation/~$-Crash-Data-Code-Guide-2003-To-2015.docx


Okay so here are somethings that we can do - some questions we can do:

1. What age is the most represented, what age is the most over represented

2. What ACC TIME bucket is the most prone? Challenge, waht ACC Time is the most overprone?

3. What city has the most accidents?

6. What percentage of accidents are given the alchool test?

7. What are the most common primary secondar and tertiarry contributin factors and how represented are they (like what perecentage is the number one factor and number two factor) - could help in picking a direction for what to do to help reduce crashes the most

8. What part of the car is the most damaged?

9. What is the ususal crash configuration? - what disagram?

10. At what kind of light do crashses usually occour, is it really true that nightime crashes are more often? Are they more dangerous? Is the speed higher at one timezone compared to other?

11. Any make overrepresented in crashes?

12. what's the maximum number of deaths in a crash?

13. Do more alcholics crash or non alcholics?

14. Suset vs Sunrise crashes and roadway direction - any changes in that?

15. Any vehical types that are overrepresented?

16. Any changes when snowing - WEATHER WEATH2 use that to track for snow and rain - what percentage of MN crashes occour in snow and are they more dangeorous than non snow - in terms of accident severatiy


Other cool things that could be done reagrading map analaysis - 

1. Track all the accidents that occour when drinking alchool - roads that people who drink alchool usually take or is over represented

2. Do analysis to find locations that have more accidents in snow than other weather events, might tell us a bit more about roads and intersections that need more work

3. Could there be a mapping algorithm uses for weather (and possibly for drunk drivers)?


Other observations - we get about 80,000 accidents each year accept 2020, where it was 50k - so total accidents that we have access to is around 450k or around half a million data points. That's a lot. We can also request data for 2022 and 2023 soon and add around 150k more accidents to our dataset. 

Population of MN is about 5.5 Million and has been increasing, perhaps we can do a population vs accients metric to see if the percentageg of people who get into an accident is consistent or not. Perhaps it will be. 

80k/5.5m = =8/550 or ~= 1.5%

It would also be intresting to see how this figure compares against different states and such



In [3]:
import pandas as pd

# Function to load the data for a given year
def load_data(year):
    base_path = f"/Users/yd211/Documents/GitHub/DataSet/DVSCrashData/MN Crash Data {year}"
    
    # Define file paths
    acc_loc = f"{base_path}/mn-{year}-acc.txt"
    per_loc = f"{base_path}/mn-{year}-per.txt"
    veh_loc = f"{base_path}/mn-{year}-veh.txt"
    
    # Read the data files
    # We use low_memory=False to avoid the dtype warning for mixed types
    accidents = pd.read_csv(acc_loc, delimiter='\t', low_memory=False)
    people = pd.read_csv(per_loc, delimiter='\t', low_memory=False)
    vehicles = pd.read_csv(veh_loc, delimiter='\t', low_memory=False)
    
    return accidents, people, vehicles

# Function to merge dataframes for a given year
def merge_data(accidents, people, vehicles):
    # Merge dataframes on 'ACCN' column
    merged_df = pd.merge(accidents, people, on='ACCN', how='outer')
    merged_df = pd.merge(merged_df, vehicles, on='ACCN', how='outer')

    return merged_df

# Function to load and merge data for multiple years
def load_and_merge_years(years):
    merged_yearly_data = []

    for year in years:
        accidents, people, vehicles = load_data(year)
        merged_df = merge_data(accidents, people, vehicles)
        merged_yearly_data.append(merged_df)
    
    # Concatenate all yearly dataframes
    all_years_merged_df = pd.concat(merged_yearly_data, ignore_index=True)
    return all_years_merged_df

# Main function
def main(years):
    # Load and merge data for the given years
    merged_df = load_and_merge_years(years)
    
    # Print some basic info
    print("Number of rows in merged data:", len(merged_df))
    print("Columns in the merged data:", merged_df.columns.tolist())

    # Save merged data to a CSV file
    years_str = "_".join(map(str, years))
    merged_df.to_csv(f"merged_data.csv", index=False)
    print(f"Merged data saved to merged_data_{years_str}.csv")


years = [2016,2017,2020,2021]  # Change this to the list of years you want to analyze
main(years)
years_str = "_".join(map(str, years))
CSV_path = "merged_data.csv"


Number of rows in merged data: 1257598
Columns in the merged data: ['ACCN', 'AGENCY', 'LOCCASE', 'HITRUN', 'PROPDAM', 'NUMMV', 'NUMFAT', 'NUMINJ', 'DOLMIN', 'ACCDATE', 'COUNTY', 'CITYTWP', 'CITYNAME', 'ACCTYPE', 'SBUS', 'LOCFHE', 'BRIDGE', 'WKZNTYPE', 'LOCWKZN', 'WORKERS', 'RDSURF', 'INTREL', 'WEATHER1', 'WEATHER2', 'LIGHT', 'DIAGRAM', 'OFFTYPE', 'INTERSECT', 'ACCSEV', 'CFR1', 'CFR2', 'FATAL', 'FATWKZN', 'INJURY', 'INTYPE', 'LANDOWN', 'LEPRES', 'NUMNM', 'ONROAD', 'RTSYS', 'WKZNREL', 'CITY', 'XCOORD', 'YCOORD', 'TOWNSHIP', 'ACCTIME', 'URBRURT', 'ACCDAY', 'ACCYEAR', 'RVN_x', 'nmaction', 'unitper', 'RPN', 'POSITN', 'DLSTATE', 'DLCLASS', 'DLSTAT', 'VIOLS', 'DLREST1', 'PHYSCND1', 'RECOMND', 'SEX', 'SAFEQP1', 'AIRBAG', 'EJECT', 'INJSEV', 'ALCTEST', 'ALCTYPE', 'DRUGTEST', 'DRUGTYPE', 'METHHOSP', 'DLZIP', 'PTYPE', 'DISTRACT', 'ALCSUSP', 'NEWBAC', 'SPEEDING', 'CFP1', 'CFP2', 'CFP3', 'CFP4', 'CHARGED', 'DLENDOR1', 'DLENDOR2', 'DLENDOR3', 'DLJURIS', 'DLREST2', 'DLREST3', 'DRUGRES', 'DRUGSUSP', 'N

In [4]:
merged_df = pd.read_csv(CSV_path, low_memory=False)

print("done")

done


In [9]:
# Filter out invalid ages (e.g., 999)
valid_ages = merged_df[merged_df['AGE'] != 999]

# Get the top 10 most represented ages
top_10_ages = valid_ages['AGE'].value_counts().head(10)
print("Top 10 most represented ages:")
print(top_10_ages)


Top 10 most represented ages:
AGE
18.0    33691
17.0    33137
19.0    32209
20.0    31580
21.0    30341
22.0    30067
16.0    29800
23.0    29454
24.0    28531
25.0    27289
Name: count, dtype: int64


In [4]:
# Print the top 5 most represented ACC TIME values
top_5_times = merged_df['ACCTIME'].value_counts().head(5)
print("Top 5 most represented ACC TIME values:")
print(top_5_times)


Top 5 most represented ACC TIME values:
ACCTIME
1700    7457
1600    7075
1730    6268
1530    6136
1500    6006
Name: count, dtype: int64


In [5]:
# Question 3: What city has the most accidents?
most_accidents_city = merged_df['CITY'].value_counts().idxmax()
print(f"The city with the most accidents is: {most_accidents_city}")

# Also, let's print the top 5 cities with the most accidents
top_5_cities = merged_df['CITY'].value_counts().head(5)
print("Top 5 cities with the most accidents:")
print(top_5_cities)


The city with the most accidents is: 2585
Top 5 cities with the most accidents:
CITY
2585    179060
3425     92084
3235     31108
3380     30200
1040     28385
Name: count, dtype: int64


2585= Minneapolis

3425=ST PAUL

1040 = DULUTH 

3235 = ROCHESTER

3380 = ST CLOUD 

Intresting to see Minneapolist and St.Paul as no 1 no 2, it's validating for sure. Another intresting thing we could do is divide this number by population and see what data we get

In [6]:
# Question 6: What percentage of accidents are given the alcohol test?

# Filter out the rows where ALCTEST is relevant (i.e., rows related to people)
people_with_alcohol_test_info = merged_df.dropna(subset=['ALCTEST'])

# Calculate the percentage of people given an alcohol test
alcohol_test_given = people_with_alcohol_test_info[people_with_alcohol_test_info['ALCTEST'] == 1]

# Calculate the percentage
percentage_alcohol_test_given = (len(alcohol_test_given) / len(people_with_alcohol_test_info)) * 100
print(f"The percentage of accidents where an alcohol test was given to at least one person is: {percentage_alcohol_test_given:.2f}%")


The percentage of accidents where an alcohol test was given to at least one person is: 2.53%


In [7]:
# Question 7: Most common primary, secondary, and tertiary contributing factors

# Most common primary contributing factor
primary_factors = merged_df['CFP1'].value_counts().head(5)
print("Top 5 primary contributing factors:")
print(primary_factors)

# Most common secondary contributing factor
secondary_factors = merged_df['CFP2'].value_counts().head(5)
print("Top 5 secondary contributing factors:")
print(secondary_factors)

# Most common tertiary contributing factor
tertiary_factors = merged_df['CFP3'].value_counts().head(5)
print("Top 5 tertiary contributing factors:")
print(tertiary_factors)


Top 5 primary contributing factors:
CFP1
1.0     644275
2.0      90023
99.0     85247
4.0      84418
90.0     58523
Name: count, dtype: int64
Top 5 secondary contributing factors:
CFP2
90.0    10862
4.0      9582
2.0      7751
70.0     7368
68.0     7270
Name: count, dtype: int64
Top 5 tertiary contributing factors:
CFP3
90.0    2189
62.0    2140
68.0    1485
72.0    1449
70.0    1359
Name: count, dtype: int64


### Top 5 Primary Contributing Factors
- **1.0**: No Clear Contributing Factor
- **2.0**: Fail to Yield Right of Way
- **4.0**: Following Too Closely
- **70.0**: Other Contributing Factor
- **99.0**: Unknown

### Top 5 Secondary Contributing Factors
- **90.0**: Other Contributing Factor
- **4.0**: Following Too Closely
- **1.0**: No Clear Contributing Factor
- **2.0**: Fail to Yield Right of Way
- **74.0**: Other Human Contributing Factor

### Top 5 Tertiary Contributing Factors
- **62.0**: Weather
- **90.0**: Other Contributing Factor
- **72.0**: Driver Inattention/Distraction
- **68.0**: Other Vision Related Factor
- **75.0**: Driver on Phone/CB/Radio


Intresting how phone is not the priamry reason for accidents it seems like or may be it is, just not represnetd that well

In [11]:
# Question 8: What part of the car is the most damaged?
most_damaged_part = merged_df['DAMAREA'].value_counts().idxmax()
print(f"The part of the car that is most damaged is: {most_damaged_part}")

# Also, let's print the top 5 most damaged parts
top_5_damaged_parts = merged_df['DAMAREA'].value_counts().head(5)
print("Top 5 most damaged parts of the car:")
print(top_5_damaged_parts)


The part of the car that is most damaged is: 10.0
Top 5 most damaged parts of the car:
DAMAREA
10.0    492995
16.0    281667
21.0     87418
11.0     75496
17.0     38416
Name: count, dtype: int64


### Top 5 Most Damaged Parts of the Car

- **10.0**: Bottom (Undercarriage)
- **16.0**: Multiple Areas
- **21.0**: Other
- **11.0**: Multiple Areas
- **3.0**: Right Center

In [9]:
# Question 9: What is the usual crash configuration (diagram)?
most_common_diagram = merged_df['DIAGRAM'].value_counts().idxmax()
print(f"The most common crash configuration (diagram) is: {most_common_diagram}")

# Also, let's print the top 5 most common crash configurations
top_5_diagrams = merged_df['DIAGRAM'].value_counts().head(5)
print("Top 5 most common crash configurations (diagrams):")
print(top_5_diagrams)


The most common crash configuration (diagram) is: 12.0
Top 5 most common crash configurations (diagrams):
DIAGRAM
12.0    517521
5.0     291045
10.0    130205
13.0     57821
90.0     42188
Name: count, dtype: int64


### Top 5 Most Common Crash Configurations (Diagrams)

- **12.0**: Unknown (Not listed in the provided codes)
- **5.0**: Right Angle
- **10.0**: Head-On
- **13.0**: Unknown (Not listed in the provided codes)
- **90.0**: Other


In [10]:
max_deaths = merged_df['NUMFAT'].value_counts()
print(f"The deaths are: {max_deaths}")

The deaths are: NUMFAT
0    1250139
1       6633
2        697
3        103
4         26
Name: count, dtype: int64


1. Failed to yeild wrong way, find hotspots where the labeling is bad
2. See alchoool deaths vs alchool tests vs all tests, are there roads athat alcholics tend to frequent
3. winter accidents vs regular accidents - does the severtiy change for some roads, like roads that are very slippry in winter
4. Sunrise vs sunsets, roads and times of years where the sune will be your enemy while driving 
5. How many times are the same poeple represented - what's the most number of times a person has been in an accident (not sure if we can do this)
6. Different routing algorithms for summer vs winter, safe routs vs fast routes, etc. 

# ideas for safe routes

Ways to approach it - 

1. Create a dictionary of clusters of accidents and times amd weatehr events where they are the most represented, and than use those to basically block certian routes. Than find the fastest road from the blocked routes.

So basically the way to find the safest routes, is use the gmap api, but just create blocks on certain parts of the roads wehre a lot of accidents happen at certain time of day and times of year 